**Project 2 Phase 2**<br>
Group 8
<br>
<table style="width:100%">
  <tr>
    <th style = "text-align: left">#</th>
    <th style = "text-align: left">Name</th>
    <th style = "text-align: left">Lastname</th>
    <th style = "text-align: left">Matr Number</th>
  </tr>
  <tr>
    <td style = "text-align: left">1</td>
    <td style = "text-align: left">Christian</td>
    <td style = "text-align: left">Peinthor</td>
    <td style = "text-align: left">11815592</td>
  </tr>
  <tr>
    <td style = "text-align: left">2</td>
    <td style = "text-align: left">Michael</td>
    <td style = "text-align: left">Weikl</td>
    <td style = "text-align: left">1154652</td>
  </tr>
  <tr>
    <td style = "text-align: left">3</td>
    <td style = "text-align: left">Uros</td>
    <td style = "text-align: left">Zivanovic</td>
    <td style = "text-align: left">12032271</td>
  </tr>
  <tr>
    <td style = "text-align: left">4</td>
    <td style = "text-align: left">Maximilian</td>
    <td style = "text-align: left">Hofmann</td>
    <td style = "text-align: left">12019987</td>
  </tr>
  
</table>
<br>

# Basic stuff

In [1]:
from typing import Union

import numpy as np
import numpy.linalg as lin
import scipy.optimize

In [2]:
class Problem():
    def f(self, x):
        raise NotImplementedError("f() is not implemented for this Problem")
    
    def grad_f(self, x):
        raise NotImplementedError("grad_f() is not implemented for this Problem")
            
    def hessian(self, x):
        raise NotImplementedError("hessian() is not implemented for this Problem")      
        

In [3]:
class PolynomialProblem(Problem):
        @property
        def actual_minima(self):
            raise NotImplementedError("actual_minima() is not implemented for this Problem")      


############################################################
#                Polynomial definitions
############################################################

class Rosenbrock(PolynomialProblem):
    def f(self, x):
        return 100*(x[1] - x[0]**2)**2 + (1 - x[0])**2

    def grad_f(self, x):
        return np.array([-400*x[0]*(x[1] - x[0]**2) - 2*(1 - x[0]),
                         200*(x[1] - x[0]**2)])

    def hessian(self, x):
        return np.array([[-400*(x[1] - 3*x[0]**2) + 2, 
                          -400*x[0]],
                         [-400*x[0], 
                          200]])
    
    @property
    def actual_minima(self):
        return np.array([1,1])


#this is the function below rosenbrock defined in the project description
class Function2(PolynomialProblem):
    def f(self, x):
        return 150*(x[0] * x[1])**2 + (0.5 * x[0] + 2 * x[1] - 2)**2
    
    def grad_f(self, x):
        return np.array([x[0] * (300 * x[1]**2 + 0.5) + 2 * x[1] - 2, 300*x[0]**2 * x[1] + 2 * x[0] + 8 * x[1] - 8])

    def hessian(self, x):
        return np.array([[300 * x[1]**2 + 0.5, 600 * x[0]*x[1] + 2],[600 * x[0]*x[1] + 2, 300 * x[0]**2 + 8]])

    #self calculated minima
    @property
    def actual_minima(self):
        return np.array([[0,1],[4,0]])

# Implementations

## Simplex

In [21]:
def simplex_method(linear_problem: Union[np.ndarray, None], constraints: np.ndarray, constraint_rhs: np.ndarray):
    def swap(variables: list, exits_idx: int, enters_idx: int) -> None:
        """
        In-/excluding respective vectors into/from the basis.
        """
        tmp = variables[exits_idx]
        variables[exits_idx] = variables[enters_idx]
        variables[enters_idx] = tmp
    
    constraint_rhs = constraint_rhs.reshape((-1, 1))
    n_constraints = constraints.shape[0]  # Number of basis vectors
    n_orig_variables = constraints.shape[1]  # Number of original_variables in each constraint
    n_variables = constraints.shape[1]  # Number of variables in each constraint

    tableau = np.concatenate((constraints, np.eye(n_constraints)), axis=1)  # Extended matrix of the system by slack constraints
    x = np.concatenate((np.zeros([n_variables, 1]), constraint_rhs))  # Right parts of the constraints
    
    non_base_weigths = np.zeros((n_variables, 1)) if linear_problem is None else linear_problem
    base_weights = np.ones((n_constraints, 1)) if linear_problem is None else np.zeros((n_constraints, 1))
    weights = np.vstack((non_base_weigths, base_weights))
    
    variable_idx = list(range(len(weights)))  # Indices for all variables
    priv_x_B, x_B = None, None


    while True:
        basis_idx = variable_idx[n_variables:]
        basis = tableau.T[basis_idx].T
        basis_inv = lin.inv(basis)
        priv_x_B = x_B
        x_B = x[basis_idx]
        c_B = weights[basis_idx]
        lam = lin.inv(basis.T) @ c_B

        non_basis_idx = variable_idx[:n_variables]
        c_N = weights[non_basis_idx]
        n_T = tableau.T[non_basis_idx]
        s_N = c_N - (n_T @ lam)

        if np.all(s_N >= 0) or (x_B is not None and priv_x_B is not None and np.sum(priv_x_B - x_B) == 0):
            print("Optimal solution found!!!")
            solution = None if linear_problem is None else c_B.T @ basis_inv @ constraint_rhs
            return x[:n_orig_variables].flatten(), x[n_orig_variables:].flatten(), solution

        # since here must be at least one s_q < 0, we take the argmin to get the index of the smallest value.
        # q_idx = np.argmin(s_N)  # Fetch the idx in the non basis idx set
        # q = non_basis_idx[q_idx]  # Fetch the variable_idx
        # d = basis_inv @ tableau.T[q].T
        # d = np.reshape(d, (-1, 1))

        d = basis_inv @ tableau
        n_q = np.vstack((-d.T[:n_variables].T, np.identity(n_variables)))

        change = (weights.T @ n_q) / np.linalg.norm(n_q, axis=0)
        change[(s_N >= 0).T] = float('inf')
        q_idx = np.argmin(change)
        q = non_basis_idx[q_idx]

        if np.all(d <= 0):
            print("Problem is unbound!!!")
            return None, None, None
        d_q = np.reshape(d[:, q], (-1, 1))
        piv_weigths = x_B / d_q
        p_idx = np.argmin(piv_weigths)
        x_q = piv_weigths[p_idx]
        x[basis_idx] = x_B - (d_q * x_q)
        x[q] = x_q  # set entering variable vaLue

        # In-/excluding respective vectors into/from the basis:
        swap(variables=variable_idx, exits_idx=p_idx + n_variables, enters_idx=q_idx)

## Active Set

In [5]:
def simplex(set_b, set_n, A, b, c, print_steps=False):

    A = A.T
    c = np.reshape(c, (-1, 1))
    
    xN = np.zeros((len(set_n), 1))
    x = np.zeros((A.shape[0], 1))
    
    B = A[set_b].T
    cB = c[set_b]
    
    N = A[set_n].T
    cN = c[set_n]

    xB = np.linalg.inv(B) @ b
    
    for j, i in enumerate(set_b):
        x[i] = xB[j]

    while True:

        l = np.linalg.inv(B).T @ cB

        sN = cN - N.T @ l
        
        if print_steps:
            print(f'\n\nxB:\n{xB}\n\nsN:\n{sN}\n\nlambda:\n{l}\n\nf(x):\n{f(x[:2])}')

        if all(s >= 0 for s in sN):
            optimal_points = [p[0] for p in x]
            return optimal_points

        q = None
        min_change = np.inf
        d = None
        for i, k in enumerate(set_n):

            d_q = np.linalg.inv(B) @ np.reshape(A[k], (-1, 1))
            e = np.identity(c.shape[0] - d_q.shape[0])
            n_q = np.vstack((-d_q, np.reshape(e[i], (-1, 1))))

            change = (c.T @ n_q) / np.linalg.norm(n_q)

            if change < min_change and sN[i] < 0:
                min_change = change
                q = k
                d = d_q

        if all(d_i <= 0 for d_i in d):
            return False

        p = None
        minimum = np.inf
        for i in range(len(d)):
            if d[i] > 0:
                if xB[i] / d[i] < minimum:
                    minimum = xB[i] / d[i]
                    p = i

        x[q] = minimum
              
        xB = xB - d * x[q]
        
        for j, i in enumerate(set_b):

            x[i] = xB[j]

        xN[set_n.index(q)] = x[q]

        for j, i in enumerate(set_n):

            x[i] = xN[j]

        index_n = set_b[p]

        set_b[p] = q
        set_n[set_n.index(q)] = index_n

        for j, i in enumerate(set_b):
            B[:, j] = A[i]

            cB[j] = c[i]
            xB[j] = x[i]

        for j, i in enumerate(set_n):
            N[:, j] = A[i]

            cN[j] = c[i]

def find_feasible_point(A, b, n_constraints, n_variables):
    
    # modifying matrices for simplex method
    A_simp = np.hstack((A, np.identity(n_constraints)))
    b_simp = np.reshape(b, (-1, 1))
    c_simp = np.ones((n_constraints, 1))
    c_simp = np.vstack((np.zeros((n_variables, 1)), c_simp))

    set_b = [i for i in range(A_simp.shape[1] - n_constraints, A_simp.shape[1])]
    set_n = [i for i in range(A_simp.shape[1] - n_constraints)]

    minimiser = simplex(set_b, set_n, A_simp, b_simp, c_simp)

    x = minimiser[:n_variables]
    z = minimiser[n_variables:]
    
    W = np.where(A @ x == b)[0].tolist()
    
    return x, W

In [15]:
def compute_learning_rate(A, p, x, w, b):
    numerator = b - (A @ x)
    denominator = A @ p
    possible_alphas = numerator / denominator

    idx_gtz = np.where(denominator > 0)[0].tolist()

    if len(idx_gtz) == 0:
        return 1, None
    else:
        blocking = []
        for idx in idx_gtz:
            if idx not in w:
                blocking.append(idx)

        if len(blocking) == 0:
            return 1, None
        else:
            possible_alphas = possible_alphas[blocking]
            min_alpha = np.argmin(possible_alphas)
            if len(min_alpha.shape) != 0:
                min_alpha = min_alpha[0]

            if possible_alphas[min_alpha] < 1:
                return possible_alphas[min_alpha], min_alpha
            else:
                return 1, None


def compute_descent_direction_and_lagrangien(G, x, c, A, w):
    working_A = A[w]
    size_of_zeros = working_A.shape[0]

    problem = np.concatenate((
        np.concatenate((G, working_A.T), axis=1),
        np.concatenate((working_A, np.zeros((size_of_zeros, size_of_zeros))), axis=1)),
        axis=0)
    g_k = (G @ x) + c
    rhs = np.concatenate((-g_k, np.zeros(size_of_zeros)), axis=0)

    result = np.linalg.solve(problem, rhs.T)

    descent_direction = result[0:len(G)]
    lagrangiens = result[len(G):]

    return descent_direction, lagrangiens


def active_set_method(G, c, A, b, x0=None, steps=1000):
    if x0 is None:
        x, _, _ = simplex_method(None, A, b)
        print(f"No starting point given: Found one feasible point at '{x}'")
    else:
        x = np.copy(x0)
        
    w: list = np.where(A @ x == b)[0].tolist()

    for k in range(steps + 1):
        p, lambdas = compute_descent_direction_and_lagrangien(G=G, x=x, c=c, A=A, w=w)

        if np.all(np.abs(p) <= 1e-15):
            if np.all(lambdas >= 0):
                return x, k
            else:
                j = np.argmin(lambdas)
                w_j = w[j]
                while w_j in w:
                    w.remove(w_j)

        else:
            alpha, blocking_idx = compute_learning_rate(A, p, x, w, b)
            x = x + alpha * p

            # Are there blocking constraints
            # http://lendek.net/teaching/OPT/qp.pdf
            if alpha < 1:
                w.append(blocking_idx)

    return None, k

## SQP

In [7]:
### Implementation

## Derivative Approximations

In [8]:
def e_i(size, index):
    arr = np.zeros(size)
    arr[index] = 1.0
    return arr


# these return the function that computes the gradient, which can then in turn be called to compute the gradient
def approx_grad(f, e=1.1e-8):
    def grad_f(x):
        if x.size == 1:
            return (f(x + e) - f(x)) / e
        return np.array([(f(x + e * e_i(x.size, i)) - f(x)) / e for i in range(x.size)])

    return grad_f


def approx_hessian(f, e=1.1e-8):
    def hessian_f(x):
        if x.size == 1:
            return (f(x + 2 * e) - 2 * f(x + e) + f(x)) / e ** 2
        return np.array([[(f(x + e * e_i(x.size, i) + e * e_i(x.size, j)) - f(
            x + e * e_i(x.size, i)) - f(x + e * e_i(x.size, j)) + f(
            x)) / e ** 2 for j in range(x.size)] for i in range(x.size)])

    return hessian_f

# Testruns

## Simplex

In [22]:
prob = np.array([[-3], [-2]])
const = np.array([[1, 1], [2, 0.5]])
rhs = np.array([[5], [8]])

x_val, slack_val, sol = simplex_method(linear_problem=prob, constraints=const, constraint_rhs=rhs)
print(f"x = {x_val}")
print(f"slack = {slack_val}")
print(f"solution = {sol}")

Optimal solution found!!!
x = [3.66666667 1.33333333]
slack = [0. 0.]
solution = [[-13.66666667]]


## Generating and solving simplex problems

In [10]:
def generate_problem(n_variables, n_constraints):
    A = np.random.randint(-10, 11, size=(n_constraints, n_variables))
    c = np.random.randint(-10, 11, size=(n_variables, 1))
    b = np.random.randint(0, 11, size=(n_constraints, 1))

    return A, b, c

In [11]:
def generate_and_solve(n_problems, n_variables, n_constraints):
    for i in range(n_problems):

        const, rhs, prob = generate_problem(n_variables, n_constraints)

        x_val, slack_val, sol = simplex_method(linear_problem=prob, constraints=const, constraint_rhs=rhs)
        
        while sol is None:
            const, rhs, prob = generate_problem(n_variables, n_constraints)
            x_val, sol = simplex_method(linear_problem=prob, constraints=const, constraint_rhs=rhs)

        print(f'A:\n{const}\nb:\n{rhs}\nc:\n{prob}\n')
        print(f'Minimum f(x)={sol} at:\n{x_val}\n\n')

In [23]:
generate_and_solve(n_problems=5, n_variables=10, n_constraints=5)

KeyboardInterrupt: 

## Active Set

In [24]:
G = np.array([[-2, 0], [0, -2]])
c = np.array([6, 4])
A = np.array([[1, 1], [-1, 0], [0, -1]])
b = np.array([3, 0, 0])

# feasible_x, W = find_feasible_point(A, b, n_constraints=3, n_variables=2)
# print(feasible_x, W)

result, iterations = active_set_method(G, c, A, b)
print(f"Found solution at {result} after {iterations} iterations")

[0.0, 0.0] [1, 2]
Optimal solution found!!!
No starting point given: Found one feasible point at '[0. 0.]'
Found solution at [0. 0.] after 0 iterations


## SQP